<a href="https://colab.research.google.com/github/chloebs4590/Metis-Engineering/blob/main/train_emissions_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description of Notebook
This notebook is for train emissions data collection

In [1]:
reset -fs

In [2]:
!pip install googlemaps

  Created wheel for googlemaps: filename=googlemaps-4.6.0-py3-none-any.whl size=38554 sha256=373e9fabea009d80d97b8950fd877e1750578e3d02a7322ec4d6e5b079a1f6fe
  Stored in directory: /root/.cache/pip/wheels/80/db/c0/6d958585fa97b20e250bf437acf7e6e715b4809c2dd4e55367
Successfully built googlemaps


In [47]:
import googlemaps
import pandas as pd
import os
import time
import pickle
import requests
import json

In [4]:
# mount Google Drive
from google.colab import drive # import drive from google colab
from os.path import join
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [6]:
os.getcwd()

'/content'

In [7]:
os.chdir('/content/drive/MyDrive/Data Science Metis/Engineering')

Read in train station spreadsheets

In [8]:
worksheet = gc.open('Amtrak Routes and Train Stations_rev').sheet1
rows = worksheet.get_all_values()
routes_stations = pd.DataFrame.from_records(rows[1:])

In [9]:
routes_stations.columns = ['route','station_name']

In [10]:
routes_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   route         816 non-null    object
 1   station_name  816 non-null    object
dtypes: object(2)
memory usage: 12.9+ KB


In [11]:
routes_stations.head()

,route,station_name
0,Acela,"Boston, MA - South Station (BOS)"
1,Acela,"Boston, MA - Back Bay Station (BBY)"
2,Acela,"Route 128, MA (RTE)"
3,Acela,"Providence, RI - Amtrak/MBTA Station (PVD)"
4,Acela,"New Haven, CT - Union Station (NHV)"


In [12]:
# read in stations_locations_geocoded

with open('stations_locations_geocoded.pkl','rb') as fid:
  stations_locations = pickle.load(fid)

In [13]:
# convert latitude and longitude columns to numeric

stations_locations[['latitude', 'longitude']] = stations_locations[['latitude', 'longitude']].apply(pd.to_numeric)

In [14]:
# merge latitude and longitude columns

stations_locations['coordinates'] = stations_locations[['latitude', 'longitude']].apply(tuple, axis=1)

In [15]:
# keep only necessary columns

stations_locations = stations_locations.drop(columns=['longitude', 'latitude','objectid_1','objectid','station_descripton',
                                                      'bus_or_train','address_2','name'])

In [16]:
stations_locations = stations_locations.drop(columns=['x', 'y'])

In [17]:
stations_locations.head()

,zip_code,state,city,address_1,code,station_name,state_rev,full_address,coordinates
0,21001,MD,Aberdeen,18 East Bel Air Avenue,ABE,"Aberdeen, MD",Maryland,"18 East Bel Air Avenue, Aberdeen, Maryland 21001","(39.5084117, -76.163203)"
1,12144,NY,Rensselaer,525 East Street,ALB,"Albany-Rensselaer, NY",New York,"525 East Street, Rensselaer, New York 12144","(42.6413525, -73.7411492)"
2,97321,OR,Albany,110 10th Avenue SW,ALY,"Albany, OR",Oregon,"110 10th Avenue SW, Albany, Oregon 97321","(44.6301042, -123.1029538)"
3,49224,MI,Albion,300 North Eaton Street,ALI,"Albion, MI",Michigan,"300 North Eaton Street, Albion, Michigan 49224","(42.2472581, -84.7556711)"
4,87102,NM,Albuquerque,320 1st Street SW,ABQ,"Albuquerque, NM",New Mexico,"320 1st Street SW, Albuquerque, New Mexico 87102","(35.0819655, -106.6476898)"


In [18]:
stations_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   zip_code      540 non-null    object
 1   state         540 non-null    object
 2   city          540 non-null    object
 3   address_1     540 non-null    object
 4   code          540 non-null    object
 5   station_name  540 non-null    object
 6   state_rev     540 non-null    object
 7   full_address  540 non-null    object
 8   coordinates   540 non-null    object
dtypes: object(9)
memory usage: 38.1+ KB


Clean up routes_stations for merging with stations_locations

In [19]:
# create code column

def create_code_col(x):
  code = x[-5:]
  code = code[1:-1]
  return code

routes_stations['code'] = routes_stations['station_name'].map(create_code_col)

In [20]:
# remove code from station_name column

routes_stations['station_name'] = routes_stations['station_name'].map(lambda x: x[:-6])

In [21]:
routes_stations.head()

,route,station_name,code
0,Acela,"Boston, MA - South Station",BOS
1,Acela,"Boston, MA - Back Bay Station",BBY
2,Acela,"Route 128, MA",RTE
3,Acela,"Providence, RI - Amtrak/MBTA Station",PVD
4,Acela,"New Haven, CT - Union Station",NHV


Left merge stations_locations on routes_stations

In [22]:
routes_stations_locations = routes_stations.merge(stations_locations, how='left', on='code',suffixes=('_route', '_location'))

In [23]:
routes_stations_locations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 816 entries, 0 to 815
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   route                  816 non-null    object
 1   station_name_route     816 non-null    object
 2   code                   816 non-null    object
 3   zip_code               816 non-null    object
 4   state                  816 non-null    object
 5   city                   816 non-null    object
 6   address_1              816 non-null    object
 7   station_name_location  816 non-null    object
 8   state_rev              816 non-null    object
 9   full_address           816 non-null    object
 10  coordinates            816 non-null    object
dtypes: object(11)
memory usage: 76.5+ KB


In [24]:
routes_stations_locations.head(2)

,route,station_name_route,code,zip_code,state,city,address_1,station_name_location,state_rev,full_address,coordinates
0,Acela,"Boston, MA - South Station",BOS,2110,MA,Boston,2 South Station,"Boston, MA",Massachusetts,"2 South Station, Boston, Massachusetts 2110","(42.348695, -71.059861)"
1,Acela,"Boston, MA - Back Bay Station",BBY,2116,MA,Boston,145 Dartmouth Street,"Boston, MA",Massachusetts,"145 Dartmouth Street, Boston, Massachusetts 2116","(42.34750349999999, -71.07493199999999)"


In [25]:
# create a column with coordinates in opposite order (longitude, latitude)

def coordinates_rev(x):
  long = x[1]
  lat = x[0]
  return list((long,lat))

routes_stations_locations['coordinates_rev'] = routes_stations_locations['coordinates'].map(lambda x:coordinates_rev(x))

In [26]:
routes_stations_locations.head(2)

,route,station_name_route,code,zip_code,state,city,address_1,station_name_location,state_rev,full_address,coordinates,coordinates_rev
0,Acela,"Boston, MA - South Station",BOS,2110,MA,Boston,2 South Station,"Boston, MA",Massachusetts,"2 South Station, Boston, Massachusetts 2110","(42.348695, -71.059861)","[-71.059861, 42.348695]"
1,Acela,"Boston, MA - Back Bay Station",BBY,2116,MA,Boston,145 Dartmouth Street,"Boston, MA",Massachusetts,"145 Dartmouth Street, Boston, Massachusetts 2116","(42.34750349999999, -71.07493199999999)","[-71.07493199999999, 42.34750349999999]"


In [27]:
# since I only really care about stations at the city level, I'll remove duplicate cities per route

print(sum(routes_stations_locations.duplicated(subset=['station_name_location', 'route'])))

routes_stations_locations = routes_stations_locations.drop_duplicates(subset=['station_name_location', 'route'])

16


In [28]:
# create a dataframe that contains routes and cities in each one

routes_locations = (routes_stations_locations.groupby('route', as_index=False)
               .agg({'station_name_location': (lambda x: list(x)), 'coordinates_rev':(lambda x: list(x))})
               .rename(columns={'station_name_location': 'locations_per_route', 'locations_coordinates':'coordinates_rev'}))

In [29]:
routes_locations.head()

,route,locations_per_route,coordinates_rev
0,Acela,"[Boston, MA, Route 128, MA, Providence, RI, Ne...","[[-71.059861, 42.348695], [-71.148665, 42.2111..."
1,Adirondack,"[New York, NY, Yonkers, NY, Croton-Harmon, NY,...","[[-73.9962784, 40.7509973], [-73.9023392, 40.9..."
2,Amtrak Cascades,"[Eugene, OR, Albany, OR, Salem, OR, Oregon Cit...","[[-123.0922546, 44.0551578], [-123.1029538, 44..."
3,Amtrak Hartford Line,"[Springfield, MA, Windsor Locks, CT, Windsor, ...","[[-72.5928997, 42.1060508], [-72.6263365, 41.9..."
4,Auto Train,"[Lorton, VA, Sanford, FL]","[[-77.2205668, 38.70843199999999], [-81.291410..."


In [30]:
# pickle routes_locations for use later

with open('routes_locations_df.pkl', 'wb') as fid:
     pickle.dump(routes_locations, fid)

In [31]:
# get number of routes
routes_stations_locations.route.nunique()

42

In [32]:
# create list of all route names
routes = list(routes_stations_locations.route.unique())
len(routes)

42

In [ ]:
routes[:2]

['Acela', 'Adirondack']

Create a dictionary where key = route and values = tuples of stations next to each other in the route and the first and last stations paired with themselves (this will make indexing and calculating down columns easier)

In [34]:
route_combos_dict = {}

for route in routes:
  route_df = routes_stations_locations.loc[routes_stations_locations.route == route]
  route_combos = []
  last_idx = len(route_df)-1
  for i in range(0, len(route_df)):
    if i == 0:
      x = route_df.code[i:i+1].values[0]
      combo = (x,x)
      route_combos.append(combo)
      combo = tuple(route_df.code[i:i + 2])
      route_combos.append(combo)
    elif i == last_idx:
      y = route_df.code[i:i+1].values[0]
      combo = (y,y)
      route_combos.append(combo)
    else:
      combo = tuple(route_df.code[i:i + 2])
      route_combos.append(combo)
  route_combos_dict[route] = route_combos

Calculate distances using Google's Distance Matrix API.
The below code comes from here: https://www.linkedin.com/pulse/calculating-distances-using-python-google-maps-r%C3%A9gis-nisengwe/?articleId=6625061973447053312

In [36]:
# create a list of lists, in which each nested list contains each station combo, including the station names, codes, coordinates and haversine distance

API_key = 'my_key'
gmaps = googlemaps.Client(key=API_key)

routes_combos_list = []

for key,value in route_combos_dict.items():
    for i in range(len(value)):
      station_1_coords = routes_stations_locations.loc[routes_stations_locations.code == value[i][0]]['coordinates'].values[0]
      station_2_coords = routes_stations_locations.loc[routes_stations_locations.code == value[i][1]]['coordinates'].values[0]

      station_1_name = routes_stations_locations.loc[routes_stations_locations.code == value[i][0]]['station_name_location'].values[0]
      station_2_name = routes_stations_locations.loc[routes_stations_locations.code == value[i][1]]['station_name_location'].values[0]

      distance = gmaps.distance_matrix(station_1_coords, station_2_coords,transit_mode='train')["rows"][0]["elements"][0]["distance"]["value"]
      routes_combos_list.append([value[i][0], station_1_name, station_1_coords, value[i][1], station_2_name, station_2_coords, distance])

In [37]:
# create dataframe

stations_combos_df = pd.DataFrame(routes_combos_list, columns = ['station_1_code','station_1_name','station_1_coords','station_2_code','station_2_name',
                                                 'station_2_coords', 'distance_meters'])
stations_combos_df['distance_mi'] = stations_combos_df['distance_meters'] / 1609

In [38]:
# drop meters column
stations_combos_df = stations_combos_df.drop(columns='distance_meters',axis=1)

In [39]:
stations_combos_df.head()

,station_1_code,station_1_name,station_1_coords,station_2_code,station_2_name,station_2_coords,distance_mi
0,BOS,"Boston, MA","(42.348695, -71.059861)",BOS,"Boston, MA","(42.348695, -71.059861)",0.000000
1,BOS,"Boston, MA","(42.348695, -71.059861)",RTE,"Route 128, MA","(42.2111905, -71.148665)",17.250466
2,RTE,"Route 128, MA","(42.2111905, -71.148665)",PVD,"Providence, RI","(41.8305099, -71.4131785)",33.078931
3,PVD,"Providence, RI","(41.8305099, -71.4131785)",NHV,"New Haven, CT","(41.2973604, -72.9267668)",103.599751
4,NHV,"New Haven, CT","(41.2973604, -72.9267668)",STM,"Stamford, CT","(41.0468938, -73.5429146)",40.182722


In [40]:
# add a column to the dataframe containing the route corresponding to each stations pair

route_combos_dict_keys = list(route_combos_dict.keys())
route_combos_dict_keys_lengths = [len(v) for v in route_combos_dict.values()]

# the below code came from here: https://stackoverflow.com/questions/48837245/how-to-multiply-a-list-of-strings-by-a-list-of-integers
routes_column = sum([[s] * n for s, n in zip(route_combos_dict_keys, route_combos_dict_keys_lengths)], [])

stations_combos_df['route'] = routes_column

In [41]:
stations_combos_df.shape

(842, 8)

In [42]:
# pickle stations_combos_df
with open('train_stations_combos_df_gmaps.pkl', 'wb') as fid:
     pickle.dump(stations_combos_df, fid)

Make all requests to Climatiq API

In [43]:
headers = {'Authorization': 'Bearer my_key','Content-type': 'application/json'}

In [44]:
distances = list(stations_combos_df.distance_mi)

In [45]:
len(distances)

842

In [48]:
responses_dict = {}

for idx, distance in enumerate(distances):
  response_dict = requests.post('https://beta2.api.climatiq.io/estimate',
                           data=json.dumps({"emission_factor": "passenger_train-route_type_intercity-fuel_source_na",\
                               "parameters": {"passengers": 1,"distance": distance,"distance_unit": "mi"}}),
                           headers=headers).json()
  responses_dict[idx] = response_dict
  time.sleep(3)

# pickle responses_dict
with open('climatiq_train_responses_dict_2_gmaps_rev.pkl', 'wb') as fid:
     pickle.dump(responses_dict, fid)